In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Reshape
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import numpy as np
from sklearn.model_selection import train_test_split

from PIL import Image
import time
import os
import argparse
import json

k.clear_session()

Using TensorFlow backend.


In [2]:
# cropping because images contained timestamps

# for folders in os.listdir('ck/img_divided'):
#     for file in os.listdir('ck/img_divided/'+folders):
#         img_arr=np.array(Image.open('ck/img_divided/'+folders+'/'+file))[12:-60]
#         img=Image.fromarray(img_arr).save('Ck/img_divided2/'+folders+'/'+file)

In [3]:
epochs = 180
nb_classes = 8
batch_size = 16

inputs={}
inputs['vgg']=[224, 224]
inputs['xception']=[299, 299]
inputs['mobilenet']=[224, 224]

classes_onehot = np.eye(8)
learn_rate = 1e-4  # sgd learning rate
momentum = .9  # sgd momentum to avoid local minimum
transformation_ratio = .15  # how aggressive will be the data augmentation/transformation

In [4]:
# xception=applications.xception.Xception(include_top=False, weights='imagenet')
# vgg=applications.vgg16.VGG16(include_top=False, weights='imagenet')
# mobilenet=applications.mobilenet.MobileNet(include_top=False, weights='imagenet')

In [5]:
def load_dataset(size, filename):
    img_array = []
    label_array = []
    with open('ck/archive/image_mapping.json') as ma:
        img_map = json.load(ma)
    for file in os.listdir('ck/' + filename):
        img=Image.open('ck/' + filename + '/' + file).convert('RGB')
        img_array.append(np.array(img).reshape(size[0], size[1], 3))
        label = img_map[file.split('.')[0]]['number']
        label_onehot = classes_onehot[label]
        label_array.append(label_onehot)
    
    return np.array(img_array), np.array(label_array)

In [13]:
def get_model(model_name):
    if model_name == 'xception':
        xception=applications.xception.Xception(include_top=False, weights='imagenet')        
        x = xception.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(128, activation='relu') (x)
        predictions = Dense(nb_classes, activation='softmax')(x)    
        model=Model(xception.input, predictions)
        
        for layer in xception.layers:
            layer.trainable = False        
    elif model_name == 'mobilenet':
        mnet = applications.mobilenet.MobileNet(include_top=False, weights='imagenet')
        x = mnet.output
        x = GlobalAveragePooling2D()(x)
        x = Reshape((none, 1, 1, 1024)) (x)
        x = Dropout() (x)
        
    return model

In [7]:
x,y=load_dataset(size = inputs['xception'], filename = 'img-xception')
x_train, x_validation,y_train, y_validation = train_test_split(x, y, test_size = 0.10, random_state = 42)

In [14]:
model=get_model('xception')

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [16]:
preprocessed_x_validation = applications.xception.preprocess_input(x_validation)

In [17]:
train_datagen = ImageDataGenerator(preprocessing_function=applications.xception.preprocess_input,
                                       rotation_range=0.35,
                                       shear_range=transformation_ratio,
                                       zoom_range=transformation_ratio,
                                       cval=transformation_ratio,
                                       horizontal_flip=True)

# validation_datagen = ImageDataGenerator(rescale=1. / 255)
# x_validation = x_validation/255

In [18]:
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=True)

# validation_generator = validation_datagen.flow(x_validation, y_validation, batch_size=16, shuffle=True)

In [15]:
tensorboard = TensorBoard(log_dir='logs-xception/{}'.format(time.time()), histogram_freq=1,
                          write_graph=True, write_images=False)

In [19]:
model.compile(optimizer=optimizers.SGD(lr=learn_rate,decay=1e-6, momentum=momentum, nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
filepath="weights-overfit/improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [21]:
model.fit_generator(train_generator,
                    nb_epoch =300,
                    validation_data = (preprocessed_x_validation, y_validation),
                    callbacks = callbacks_list)

c:\users\vibhu\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  after removing the cwd from sys.path.
c:\users\vibhu\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., callbacks=[<keras.ca..., epochs=300)`
  after removing the cwd from sys.path.


Epoch 1/300
63/63 [==============================] - 126s 2s/step - loss: 2.1459 - acc: 0.1113 - val_loss: 2.0908 - val_acc: 0.0893

Epoch 00001: val_acc improved from -inf to 0.08929, saving model to weights-overfit/improvement-01-0.09.hdf5
Epoch 2/300
63/63 [==============================] - 99s 2s/step - loss: 2.0581 - acc: 0.1182 - val_loss: 2.0450 - val_acc: 0.1250

Epoch 00002: val_acc improved from 0.08929 to 0.12500, saving model to weights-overfit/improvement-02-0.12.hdf5
Epoch 3/300
63/63 [==============================] - 99s 2s/step - loss: 2.0153 - acc: 0.1649 - val_loss: 2.0175 - val_acc: 0.1786

Epoch 00003: val_acc improved from 0.12500 to 0.17857, saving model to weights-overfit/improvement-03-0.18.hdf5
Epoch 4/300
63/63 [==============================] - 99s 2s/step - loss: 1.9811 - acc: 0.2161 - val_loss: 1.9986 - val_acc: 0.2500

Epoch 00004: val_acc improved from 0.17857 to 0.25000, saving model to weights-overfit/improvement-04-0.25.hdf5
Epoch 5/300
63/63 [=======

Epoch 87/300
63/63 [==============================] - 99s 2s/step - loss: 1.2093 - acc: 0.6163 - val_loss: 1.7961 - val_acc: 0.3571

Epoch 00087: val_acc did not improve from 0.35714
Epoch 88/300
63/63 [==============================] - 99s 2s/step - loss: 1.2099 - acc: 0.6216 - val_loss: 1.7934 - val_acc: 0.3571

Epoch 00088: val_acc did not improve from 0.35714
Epoch 89/300
63/63 [==============================] - 99s 2s/step - loss: 1.2194 - acc: 0.6272 - val_loss: 1.8014 - val_acc: 0.3482

Epoch 00089: val_acc did not improve from 0.35714
Epoch 90/300
63/63 [==============================] - 99s 2s/step - loss: 1.1833 - acc: 0.6417 - val_loss: 1.7958 - val_acc: 0.3571

Epoch 00090: val_acc did not improve from 0.35714
Epoch 91/300
63/63 [==============================] - 99s 2s/step - loss: 1.1931 - acc: 0.6403 - val_loss: 1.7900 - val_acc: 0.3571

Epoch 00091: val_acc did not improve from 0.35714
Epoch 92/300
63/63 [==============================] - 99s 2s/step - loss: 1.1845 - ac

63/63 [==============================] - 99s 2s/step - loss: 1.0162 - acc: 0.6956 - val_loss: 1.7390 - val_acc: 0.3750

Epoch 00131: val_acc did not improve from 0.37500
Epoch 132/300
63/63 [==============================] - 99s 2s/step - loss: 1.0239 - acc: 0.6938 - val_loss: 1.7489 - val_acc: 0.3750

Epoch 00132: val_acc did not improve from 0.37500
Epoch 133/300
63/63 [==============================] - 99s 2s/step - loss: 1.0540 - acc: 0.6738 - val_loss: 1.7426 - val_acc: 0.3839

Epoch 00133: val_acc improved from 0.37500 to 0.38393, saving model to weights-overfit/improvement-133-0.38.hdf5
Epoch 134/300
63/63 [==============================] - 99s 2s/step - loss: 1.0081 - acc: 0.6901 - val_loss: 1.7467 - val_acc: 0.3750

Epoch 00134: val_acc did not improve from 0.38393
Epoch 135/300
63/63 [==============================] - 99s 2s/step - loss: 1.0218 - acc: 0.6827 - val_loss: 1.7492 - val_acc: 0.3750

Epoch 00135: val_acc did not improve from 0.38393
Epoch 136/300
63/63 [==========

63/63 [==============================] - 99s 2s/step - loss: 0.9454 - acc: 0.7061 - val_loss: 1.7049 - val_acc: 0.4018

Epoch 00174: val_acc did not improve from 0.41071
Epoch 175/300
63/63 [==============================] - 99s 2s/step - loss: 0.8948 - acc: 0.7260 - val_loss: 1.6967 - val_acc: 0.4107

Epoch 00175: val_acc did not improve from 0.41071
Epoch 176/300
63/63 [==============================] - 99s 2s/step - loss: 0.9020 - acc: 0.7220 - val_loss: 1.6986 - val_acc: 0.4107

Epoch 00176: val_acc did not improve from 0.41071
Epoch 177/300
63/63 [==============================] - 99s 2s/step - loss: 0.9207 - acc: 0.7111 - val_loss: 1.7092 - val_acc: 0.4107

Epoch 00177: val_acc did not improve from 0.41071
Epoch 178/300
63/63 [==============================] - 99s 2s/step - loss: 0.8614 - acc: 0.7291 - val_loss: 1.7026 - val_acc: 0.4196

Epoch 00178: val_acc improved from 0.41071 to 0.41964, saving model to weights-overfit/improvement-178-0.42.hdf5
Epoch 179/300
63/63 [==========

63/63 [==============================] - 99s 2s/step - loss: 0.8157 - acc: 0.7363 - val_loss: 1.6800 - val_acc: 0.4018

Epoch 00218: val_acc did not improve from 0.42857
Epoch 219/300
63/63 [==============================] - 99s 2s/step - loss: 0.8254 - acc: 0.7452 - val_loss: 1.6810 - val_acc: 0.3929

Epoch 00219: val_acc did not improve from 0.42857
Epoch 220/300
63/63 [==============================] - 99s 2s/step - loss: 0.8382 - acc: 0.7312 - val_loss: 1.6741 - val_acc: 0.4196

Epoch 00220: val_acc did not improve from 0.42857
Epoch 221/300
63/63 [==============================] - 99s 2s/step - loss: 0.8217 - acc: 0.7421 - val_loss: 1.6828 - val_acc: 0.4018

Epoch 00221: val_acc did not improve from 0.42857
Epoch 222/300
63/63 [==============================] - 99s 2s/step - loss: 0.8210 - acc: 0.7450 - val_loss: 1.6822 - val_acc: 0.4018

Epoch 00222: val_acc did not improve from 0.42857
Epoch 223/300
63/63 [==============================] - 99s 2s/step - loss: 0.7851 - acc: 0.765

63/63 [==============================] - 99s 2s/step - loss: 0.7627 - acc: 0.7643 - val_loss: 1.6687 - val_acc: 0.4286

Epoch 00262: val_acc did not improve from 0.43750
Epoch 263/300
63/63 [==============================] - 99s 2s/step - loss: 0.7386 - acc: 0.7834 - val_loss: 1.6829 - val_acc: 0.4107

Epoch 00263: val_acc did not improve from 0.43750
Epoch 264/300
63/63 [==============================] - 99s 2s/step - loss: 0.7591 - acc: 0.7573 - val_loss: 1.6725 - val_acc: 0.4286

Epoch 00264: val_acc did not improve from 0.43750
Epoch 265/300
63/63 [==============================] - 99s 2s/step - loss: 0.7424 - acc: 0.7571 - val_loss: 1.6797 - val_acc: 0.4286

Epoch 00265: val_acc did not improve from 0.43750
Epoch 266/300
63/63 [==============================] - 99s 2s/step - loss: 0.7486 - acc: 0.7583 - val_loss: 1.6718 - val_acc: 0.4286

Epoch 00266: val_acc did not improve from 0.43750
Epoch 267/300
63/63 [==============================] - 99s 2s/step - loss: 0.7428 - acc: 0.766

In [22]:
model.fit_generator(train_generator,
                    nb_epoch =300,
                    validation_data = (preprocessed_x_validation, y_validation),
                    callbacks = callbacks_list)

c:\users\vibhu\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  after removing the cwd from sys.path.
c:\users\vibhu\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., callbacks=[<keras.ca..., epochs=300)`
  after removing the cwd from sys.path.


Epoch 1/300
63/63 [==============================] - 100s 2s/step - loss: 0.7222 - acc: 0.7738 - val_loss: 1.6618 - val_acc: 0.4286

Epoch 00001: val_acc did not improve from 0.43750
Epoch 2/300
63/63 [==============================] - 99s 2s/step - loss: 0.6924 - acc: 0.7851 - val_loss: 1.6590 - val_acc: 0.4196

Epoch 00002: val_acc did not improve from 0.43750
Epoch 3/300
63/63 [==============================] - 99s 2s/step - loss: 0.6871 - acc: 0.7984 - val_loss: 1.6588 - val_acc: 0.4375

Epoch 00003: val_acc did not improve from 0.43750
Epoch 4/300
63/63 [==============================] - 99s 2s/step - loss: 0.6997 - acc: 0.7930 - val_loss: 1.6568 - val_acc: 0.4375

Epoch 00004: val_acc did not improve from 0.43750
Epoch 5/300
63/63 [==============================] - 99s 2s/step - loss: 0.6936 - acc: 0.7851 - val_loss: 1.6741 - val_acc: 0.4286

Epoch 00005: val_acc did not improve from 0.43750
Epoch 6/300
63/63 [==============================] - 99s 2s/step - loss: 0.6812 - acc: 0.

63/63 [==============================] - 99s 2s/step - loss: 0.6489 - acc: 0.8133 - val_loss: 1.6751 - val_acc: 0.4554

Epoch 00045: val_acc did not improve from 0.45536
Epoch 46/300
63/63 [==============================] - 99s 2s/step - loss: 0.6422 - acc: 0.8085 - val_loss: 1.6582 - val_acc: 0.4554

Epoch 00046: val_acc did not improve from 0.45536
Epoch 47/300
63/63 [==============================] - 99s 2s/step - loss: 0.6448 - acc: 0.8022 - val_loss: 1.6619 - val_acc: 0.4554

Epoch 00047: val_acc did not improve from 0.45536
Epoch 48/300
63/63 [==============================] - 99s 2s/step - loss: 0.6358 - acc: 0.8054 - val_loss: 1.6622 - val_acc: 0.4554

Epoch 00048: val_acc did not improve from 0.45536
Epoch 49/300
63/63 [==============================] - 99s 2s/step - loss: 0.6530 - acc: 0.8004 - val_loss: 1.6617 - val_acc: 0.4554

Epoch 00049: val_acc did not improve from 0.45536
Epoch 50/300
63/63 [==============================] - 99s 2s/step - loss: 0.6534 - acc: 0.8089 - v

KeyboardInterrupt: 

In [23]:
model_json = model.to_json()
with open('model_new.json', 'w') as json_file:
    json_file.write(model_json)

In [24]:
with open('model_new.json') as file:
    a=file.read()
    model=model_from_json(a)

In [26]:
model.load_weights('weights-overfit/latest.hdf5')

In [35]:
x = np.array(Image.open('ss.png').convert('RGB').resize((299,299)))

In [36]:
a=np.expand_dims(x, axis=0)
print(a.shape)

(1, 299, 299, 3)


In [37]:
x_p=applications.xception.preprocess_input(a)

In [38]:
model.predict(x_p)

array([[0.79166955, 0.09690797, 0.00515234, 0.04810159, 0.01400502,
        0.0210832 , 0.01657484, 0.00650545]], dtype=float32)